### Main goal & Problem statement

The project will be building a stock picker with a one year time horizon that utilizes fundamental data from the companies' financial statements for selection.

The main goal would be to pick a portfolio of ten stocks that is able to outperform the market benchmark (S&P500). Regression analysis techniques will be used to determine the return predictions based on the ratios derived from the fundamental information.

A big bulk of the work will be spent on feature engineering of the fundamental data (e.g. financial and accounting ratios), so that these features will correlate with business success and be good performance indicators. Working with time series is also important as we are interested in the performance of the stock when compared to its financial performance.

### Proposed methods and models

I will work with models learnt throughout the course that show predictive ability on stock performance, and use them again for backtesting. We can use linear regression, regularized regression (Lasso or Ridge or ENet), KNN, SVM Regressor, Decision Trees as potential models. MSE will likely be the metric to assess the model performance.


Feature importance is also something to look at as we want to know which variables/ratios gives us better predictive ability for stock performance. After running these models, we can put them in a table to view the results of the top 10 selected stock portfolios.

After this, we will do backtesting to find out the volatility as well as the return profile of each portfolio. Backtesting will follow roughly these steps:
- split into train test sets and train model on training set
- for given year, use model to predict stock returns and pick 10 top performances from test set to create equal weighted portfolio
- record daily/weekly portfolio value change to see performance of portfolio
- repeat for the number of years of available data, likely using loops? (we should get return and volatility from here)

From all the models, we should get some kind of return/volatility profile, which we can then use to determine the better stocks to pick.

### Relevance

Potentially, this can be used by portfolio managers or retail investors who wish to do better than market returns without taking on too much additional risk. As I do not have any idea how the tested models would do at this point in time, I generally hope that results would at least meet the benchmark return at its minimum. The number of stocks picked is also kept small to minimize potential transaction costs.

### Timeline

I will start working on the project right after the completion of Project 4 and aim to come up with a working prototype by the 2nd week of capstone project start. I will try to refine the model in the third week and do up the presentation in the 4th week.

## Project starts here

In [1]:
%load_ext autotime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

time: 1.17 s (started: 2022-10-11 23:28:44 +08:00)


Simfin is where we will be downloading our fundamentals data from. Instead of downloading a CSV file, we will use the API key to download directly.

In [2]:
import simfin as sf

# Set your SimFin + API-key for downloading data.
sf.set_api_key('nYdjeL237mt7QaQ4OZTaPa4Xn3YECrTH')

# Set the local directory where data-files are stored.
# The directory will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

# Download the data from the SimFin server and load into a Pandas DataFrame.
# We will be getting 3 sets of financial statements (income statement, balance sheet, cashflow statement)
df_income = sf.load_income(variant='quarterly', market='us')
df_balance = sf.load_balance(variant='quarterly', market='us')
df_cashflow = sf.load_cashflow(variant='quarterly', market='us')

# Print the first rows of the data.
df_income.head()

Dataset "us-income-quarterly" on disk (3 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly" on disk (3 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (3 days old).
- Loading from disk ... Done!


SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2016-10-31      45846      USD         2016            Q4   2016-12-20   
       2017-01-31      45846      USD         2017            Q1   2017-03-08   
       2017-04-30      45846      USD         2017            Q2   2017-06-06   
       2017-07-31      45846      USD         2017            Q3   2017-09-06   
       2017-10-31      45846      USD         2017            Q4   2017-12-21   

                   Restated Date  Shares (Basic)  Shares (Diluted)  \
Ticker Report Date                                                   
A      2016-10-31     2018-12-20     324000000.0       328000000.0   
       2017-01-31     2018-03-06     322000000.0       326000000.0   
       2017-04-30     2018-05-31     321000000.0       325000000.0   
       2017-07-31     2018-08-30     321000000.0       326000000.0   
       2017-10-31     2018-12-20     324000000.0       327000000.0   

                         Revenue  Cost of Revenue  ...  \
Ticker Report Date                                 ...   
A      2016-10-31   1.111000e+09     -523000000.0  ...   
       2017-01-31   1.067000e+09     -493000000.0  ...   
       2017-04-30   1.102000e+09     -510000000.0  ...   
       2017-07-31   1.114000e+09     -518000000.0  ...   
       2017-10-31   1.189000e+09     -542000000.0  ...   

                    Non-Operating Income (Loss)  Interest Expense, Net  \
Ticker Report Date                                                       
A      2016-10-31                   -32000000.0            -16000000.0   
       2017-01-31                   -13000000.0            -16000000.0   
       2017-04-30                   -10000000.0            -15000000.0   
       2017-07-31                    -8000000.0            -13000000.0   
       2017-10-31                    -7000000.0            -13000000.0   

                    Pretax Income (Loss), Adj.  Abnormal Gains (Losses)  \
Ticker Report Date                                                        
A      2016-10-31                    151000000                      NaN   
       2017-01-31                    193000000                      NaN   
       2017-04-30                    191000000                      NaN   
       2017-07-31                    193000000                      NaN   
       2017-10-31                    226000000                      NaN   

                    Pretax Income (Loss)  Income Tax (Expense) Benefit, Net  \
Ticker Report Date                                                            
A      2016-10-31              151000000                        -25000000.0   
       2017-01-31              193000000                        -25000000.0   
       2017-04-30              191000000                        -27000000.0   
       2017-07-31              193000000                        -18000000.0   
       2017-10-31              226000000                        -49000000.0   

                    Income (Loss) from Continuing Operations  \
Ticker Report Date                                             
A      2016-10-31                                  126000000   
       2017-01-31                                  168000000   
       2017-04-30                                  164000000   
       2017-07-31                                  175000000   
       2017-10-31                                  177000000   

                    Net Extraordinary Gains (Losses)  Net Income  \
Ticker Report Date                                                 
A      2016-10-31                                NaN   126000000   
       2017-01-31                                NaN   168000000   
       2017-04-30                                NaN   164000000   
       2017-07-31                                NaN   175000000   
       2017-10-31                                NaN   177000000   

                    Net Incom

time: 1.33 s (started: 2022-10-11 23:28:46 +08:00)


In [3]:
df_balance.head()

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2016-10-31      45846      USD         2016            Q4   2016-12-20   
       2017-01-31      45846      USD         2017            Q1   2017-03-08   
       2017-04-30      45846      USD         2017            Q2   2017-06-06   
       2017-07-31      45846      USD         2017            Q3   2017-09-06   
       2017-10-31      45846      USD         2017            Q4   2017-12-21   

                   Restated Date  Shares (Basic)  Shares (Diluted)  \
Ticker Report Date                                                   
A      2016-10-31     2017-12-21     324000000.0       328000000.0   
       2017-01-31     2017-03-08     322000000.0       326000000.0   
       2017-04-30     2017-06-06     321000000.0       325000000.0   
       2017-07-31     2017-09-06     321000000.0       326000000.0   
       2017-10-31     2018-12-20     324000000.0       327000000.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Report Date                                                    
A      2016-10-31                                      2.289000e+09   
       2017-01-31                                      2.241000e+09   
       2017-04-30                                      2.389000e+09   
       2017-07-31                                      2.563000e+09   
       2017-10-31                                      2.678000e+09   

                    Accounts & Notes Receivable  ...  Short Term Debt  \
Ticker Report Date                               ...                    
A      2016-10-31                   631000000.0  ...              0.0   
       2017-01-31                   653000000.0  ...      190000000.0   
       2017-04-30                   677000000.0  ...      241000000.0   
       2017-07-31                   678000000.0  ...      280000000.0   
       2017-10-31                   724000000.0  ...      210000000.0   

                    Total Current Liabilities  Long Term Debt  \
Ticker Report Date                                              
A      2016-10-31                   945000000    1.904000e+09   
       2017-01-31                  1089000000    1.802000e+09   
       2017-04-30                  1187000000    1.802000e+09   
       2017-07-31                  1241000000    1.801000e+09   
       2017-10-31                  1263000000    1.801000e+09   

                    Total Noncurrent Liabilities  Total Liabilities  \
Ticker Report Date                                                    
A      2016-10-31                   2.603000e+09         3548000000   
       2017-01-31                   2.483000e+09         3572000000   
       2017-04-30                   2.454000e+09         3641000000   
       2017-07-31                   2.409000e+09         3650000000   
       2017-10-31                   2.328000e+09         3591000000   

                    Share Capital & Additional Paid-In Capital  \
Ticker Report Date                                               
A      2016-10-31                                 9.165000e+09   
       2017-01-31                                 5.239000e+09   
       2017-04-30                                 5.242000e+09   
       2017-07-31                                 5.285000e+09   
       2017-10-31                                 5.303000e+09   

                    Treasury Stock  Retained Earnings  Total Equity  \
Ticker Report Date                                                    
A      2016-10-31    -1.050800e+10       6.089000e+09  4.246000e+09   
       2017-01-31     0.000000e+00      -4.530000e+08  4.300000e+09   
       2017-04-30     0.000000e+00      -3.930000e+08  4.375000e+09   
       2017-07-31     0.000000e+00      -2.600000e+08  4.611000e+09   
       2017-10-31              NaN      -1.260000e+08  4.835000e+09   

                    Total Liabilitie

time: 16 ms (started: 2022-10-11 23:28:47 +08:00)


In [4]:
df_cashflow.head()

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2016-10-31      45846      USD         2016            Q4   2016-12-20   
       2017-01-31      45846      USD         2017            Q1   2017-03-08   
       2017-04-30      45846      USD         2017            Q2   2017-06-06   
       2017-07-31      45846      USD         2017            Q3   2017-09-06   
       2017-10-31      45846      USD         2017            Q4   2017-12-21   

                   Restated Date  Shares (Basic)  Shares (Diluted)  \
Ticker Report Date                                                   
A      2016-10-31     2018-12-20     324000000.0       328000000.0   
       2017-01-31     2018-03-06     322000000.0       326000000.0   
       2017-04-30     2018-05-31     321000000.0       325000000.0   
       2017-07-31     2018-08-30     321000000.0       326000000.0   
       2017-10-31     2018-12-20     324000000.0       327000000.0   

                    Net Income/Starting Line  Depreciation & Amortization  \
Ticker Report Date                                                          
A      2016-10-31                126000000.0                   56000000.0   
       2017-01-31                168000000.0                   55000000.0   
       2017-04-30                164000000.0                   54000000.0   
       2017-07-31                175000000.0                   51000000.0   
       2017-10-31                177000000.0                   52000000.0   

                    ...  Net Cash from Operating Activities  \
Ticker Report Date  ...                                       
A      2016-10-31   ...                         234000000.0   
       2017-01-31   ...                         116000000.0   
       2017-04-30   ...                         257000000.0   
       2017-07-31   ...                         228000000.0   
       2017-10-31   ...                         288000000.0   

                    Change in Fixed Assets & Intangibles  \
Ticker Report Date                                         
A      2016-10-31                            -52000000.0   
       2017-01-31                            -32000000.0   
       2017-04-30                            -43000000.0   
       2017-07-31                            -43000000.0   
       2017-10-31                            -58000000.0   

                    Net Change in Long Term Investment  \
Ticker Report Date                                       
A      2016-10-31                                  0.0   
       2017-01-31                                  NaN   
       2017-04-30                                  NaN   
       2017-07-31                                  NaN   
       2017-10-31                                  0.0   

                    Net Cash from Acquisitions & Divestitures  \
Ticker Report Date                                              
A      2016-10-31                                 -26000000.0   
       2017-01-31                                 -69000000.0   
       2017-04-30                                         0.0   
       2017-07-31                                 -57000000.0   
       2017-10-31                                         0.0   

                    Net Cash from Investing Activities  Dividends Paid  \
Ticker Report Date                                                       
A      2016-10-31                          -78000000.0     -38000000.0   
       2017-01-31                         -101000000.0     -42000000.0   
       2017-04-30                          -43000000.0     -43000000.0   
       2017-07-31                         -101000000.0     -42000000.0   
       2017-10-31                          -60000000.0     -43000000.0   

                    Cash from (Repayment of) Debt  \
Ticker Report Date                                  
A      2016-10-31                      27000000.0   
       2017-01-31

time: 16 ms (started: 2022-10-11 23:28:47 +08:00)


In [5]:
print('Income Statement CSV data is: ', df_income.shape)
print('Balance Sheet CSV data is: ', df_balance.shape)
print('Cash Flow CSV data is: ', df_cashflow.shape)

Income Statement CSV data is:  (41032, 26)
Balance Sheet CSV data is:  (41032, 28)
Cash Flow CSV data is:  (41032, 26)
time: 0 ns (started: 2022-10-11 23:28:47 +08:00)


Number of rows of data is consistent. There are altogether 36685 rows to use.

In [6]:
# Merge the data together
# Define the column features where merge takes place
list_to_merge_on = ['Ticker', 'SimFinId', 'Currency', 'Fiscal Year', 'Report Date', 'Publish Date']

# Merge the income statement and balance sheet first
merge1 = pd.merge(df_income, df_balance, on = list_to_merge_on, how = 'inner')

# Merge previous result with cashflow statement
df_merged = pd.merge(merge1, df_cashflow, on = list_to_merge_on, how = 'inner')

# Reset the index
df_merged.reset_index(inplace=True)

# Make sure that the dates are in correct format
df_merged["Report Date"] = pd.to_datetime(df_merged["Report Date"])
df_merged["Publish Date"] = pd.to_datetime(df_merged["Publish Date"])

print('Merged data matrix shape is: ', df_merged.shape)

Merged data matrix shape is:  (41032, 74)
time: 188 ms (started: 2022-10-11 23:28:47 +08:00)


In [7]:
df_merged.head()

,Ticker,Report Date,SimFinId,Currency,Fiscal Year,Fiscal Period_x,Publish Date,Restated Date_x,Shares (Basic)_x,Shares (Diluted)_x,...,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,2016-10-31,45846,USD,2016,Q4,2016-12-20,2018-12-20,324000000.0,328000000.0,...,234000000.0,-52000000.0,0.0,-26000000.0,-78000000.0,-38000000.0,27000000.0,-43000000.0,-56000000.0,90000000
1,A,2017-01-31,45846,USD,2017,Q1,2017-03-08,2018-03-06,322000000.0,326000000.0,...,116000000.0,-32000000.0,NaN,-69000000.0,-101000000.0,-42000000.0,89000000.0,-93000000.0,-58000000.0,-48000000
2,A,2017-04-30,45846,USD,2017,Q2,2017-06-06,2018-05-31,321000000.0,325000000.0,...,257000000.0,-43000000.0,NaN,0.0,-43000000.0,-43000000.0,52000000.0,-75000000.0,-67000000.0,148000000
3,A,2017-07-31,45846,USD,2017,Q3,2017-09-06,2018-08-30,321000000.0,326000000.0,...,228000000.0,-43000000.0,NaN,-57000000.0,-101000000.0,-42000000.0,39000000.0,32000000.0,29000000.0,174000000
4,A,2017-10-31,45846,USD,2017,Q4,2017-12-21,2018-12-20,324000000.0,327000000.0,...,288000000.0,-58000000.0,0.0,0.0,-60000000.0,-43000000.0,-70000000.0,8000000.0,-106000000.0,115000000


time: 31 ms (started: 2022-10-11 23:28:47 +08:00)


In [8]:
df_prices = sf.load_shareprices(variant='daily', market='us')

Dataset "us-shareprices-daily" on disk (4 days old).
- Loading from disk ... Done!
time: 9.8 s (started: 2022-10-11 23:28:47 +08:00)


In [9]:
df_prices.reset_index(inplace=True)

time: 94 ms (started: 2022-10-11 23:28:57 +08:00)


In [10]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3172869 entries, 0 to 3172868
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Ticker              object        
 1   Date                datetime64[ns]
 2   SimFinId            int64         
 3   Open                float64       
 4   Low                 float64       
 5   High                float64       
 6   Close               float64       
 7   Adj. Close          float64       
 8   Dividend            float64       
 9   Volume              int64         
 10  Shares Outstanding  float64       
dtypes: datetime64[ns](1), float64(7), int64(2), object(1)
memory usage: 266.3+ MB
time: 0 ns (started: 2022-10-11 23:28:57 +08:00)


In [11]:
df_prices.isnull().sum()

Ticker                      0
Date                        0
SimFinId                    0
Open                    21890
Low                     22342
High                    21457
Close                   21864
Adj. Close              21771
Dividend              3149382
Volume                      0
Shares Outstanding      65841
dtype: int64

time: 125 ms (started: 2022-10-11 23:28:57 +08:00)


In [12]:
df_prices.head()

,Ticker,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
0,A,2016-10-06,45846,47.01,46.88,47.43,47.30,45.09,NaN,1772801,324384755.0
1,A,2016-10-07,45846,47.33,46.99,47.49,47.38,45.16,NaN,1663127,324384755.0
2,A,2016-10-10,45846,47.58,47.55,48.15,47.69,45.46,NaN,1544958,324384755.0
3,A,2016-10-11,45846,47.19,45.42,47.19,45.69,43.55,NaN,2912537,324384755.0
4,A,2016-10-12,45846,45.78,45.42,45.81,45.47,43.34,NaN,1736076,324384755.0


time: 15 ms (started: 2022-10-11 23:28:57 +08:00)


## Define function to get price of a ticker (or find the nearest price given date)

In [14]:
# This is a function that takes in the ticker, start date and the prices dataframe
# It will return the ticker, price and date, and transaction value.

def getPrice (ticker, datestart, days_fwd, df=df_prices):
    
    # looks into a window of the next 5 days if date chosen falls on a non-trading day
    window = 5
    
    # returns all the rows between the start date and the window, filtered from df
    rows = df[
    (df["Date"].between(pd.to_datetime(datestart) + pd.Timedelta(days=days_fwd),\
                        pd.to_datetime(datestart) + pd.Timedelta(days=window + days_fwd)))\
        & (df["Ticker"]==ticker)]
    
    # if nothing between start date and window, return nothing
    # NaT stands for Not a Time
    if rows.empty:
        return [ticker, np.float("NaN"), np.datetime64('NaT'), np.float("NaN")]
    
    # else return the first row within the filtered df i.e. closest date
    # returns ticker, open price, date, volume
    else:
        return [ticker, 
                rows.iloc[0]["Open"],\
                rows.iloc[0]["Date"],\
                rows.iloc[0]["Volume"]]

time: 0 ns (started: 2022-10-11 23:29:27 +08:00)


### To demonstrate how above function works

In [15]:
# get daily AAPL data in Oct 2016
df_prices.query("Ticker=='AAPL' & Date<'2016-10-31'")

,Ticker,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
10281,AAPL,2016-10-06,111052,28.43,28.28,28.59,28.47,26.55,NaN,115117252,2.134466e+10
10282,AAPL,2016-10-07,111052,28.58,28.38,28.64,28.52,26.59,NaN,97433772,2.134466e+10
10283,AAPL,2016-10-10,111052,28.75,28.68,29.19,29.01,27.05,NaN,144943824,2.134466e+10
10284,AAPL,2016-10-11,111052,29.43,29.05,29.67,29.07,27.11,NaN,256164172,2.134466e+10
10285,AAPL,2016-10-12,111052,29.34,29.19,29.50,29.34,27.35,NaN,150347148,2.134466e+10
10286,AAPL,2016-10-13,111052,29.20,28.93,29.36,29.25,27.27,NaN,140769624,2.134466e+10
10287,AAPL,2016-10-14,111052,29.47,29.28,29.54,29.41,27.42,NaN,142608764,2.132925e+10
10288,AAPL,2016-10-17,111052,29.33,29.20,29.46,29.39,27.40,NaN,94499584,2.132925e+10
10289,AAPL,2016-10-18,111052,29.55,29.36,29.55,29.37,27.38,NaN,98213912,2.132925e+10
10290,AAPL,2016-10-19,111052,29.31,28.45,29.44,29.28,27.30,NaN,80138376,2.132925e+10


time: 188 ms (started: 2022-10-11 23:29:27 +08:00)


In [16]:
# use function to get price data from AAPL 10 Oct 2016
getPrice('AAPL', '2016-10-10', 0, df_prices)

['AAPL', 28.75, Timestamp('2016-10-10 00:00:00'), 144943824]

time: 172 ms (started: 2022-10-11 23:29:27 +08:00)


In [17]:
# use function to get price date from AAPL 10 Oct 2016
# as 09 Oct 2016 is non trading day, function will find the price info from 12 Sep instead
getPrice('AAPL', '2016-10-09', 0, df_prices)

['AAPL', 28.75, Timestamp('2016-10-10 00:00:00'), 144943824]

time: 156 ms (started: 2022-10-11 23:29:27 +08:00)


In [18]:
# use function to get APPL price date 30 days in the future using the days_adj parameter
getPrice('AAPL', '2016-09-10', 30, df_prices)

['AAPL', 28.75, Timestamp('2016-10-10 00:00:00'), 144943824]

time: 156 ms (started: 2022-10-11 23:29:27 +08:00)


**At this point, we want to combine the fundamental data and the price data together, such that we have the price data on each Report Date. However, sometimes the Report Date falls on a weekend/non-trading day, so we need to find another way to get the closest price data. Below we will attempt to do this.**

In [19]:
# all the fundemental data that we have combined so far
pd.set_option('display.max_columns', 200)
df_merged.head()

,Ticker,Report Date,SimFinId,Currency,Fiscal Year,Fiscal Period_x,Publish Date,Restated Date_x,Shares (Basic)_x,Shares (Diluted)_x,Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Depreciation & Amortization_x,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Fiscal Period_y,Restated Date_y,Shares (Basic)_y,Shares (Diluted)_y,"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Inventories,Total Current Assets,"Property, Plant & Equipment, Net",Long Term Investments & Receivables,Other Long Term Assets,Total Noncurrent Assets,Total Assets,Payables & Accruals,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Fiscal Period,Restated Date,Shares (Basic),Shares (Diluted),Net Income/Starting Line,Depreciation & Amortization_y,Non-Cash Items,Change in Working Capital,Change in Accounts Receivable,Change in Inventories,Change in Accounts Payable,Change in Other,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,2016-10-31,45846,USD,2016,Q4,2016-12-20,2018-12-20,324000000.0,328000000.0,1.111000e+09,-523000000.0,588000000.0,-405000000.0,-321000000.0,-84000000.0,NaN,183000000.0,-32000000.0,-16000000.0,151000000,NaN,151000000,-25000000.0,126000000,NaN,126000000,126000000,Q4,2017-12-21,324000000.0,328000000.0,2.289000e+09,631000000.0,533000000.0,3.635000e+09,639000000.0,135000000.0,3.385000e+09,4.159000e+09,7794000000,257000000.0,0.0,945000000,1.904000e+09,2.603000e+09,3548000000,9.165000e+09,-1.050800e+10,6.089000e+09,4.246000e+09,7794000000,Q4,2018-12-20,324000000.0,328000000.0,126000000.0,56000000.0,2000000.0,50000000.0,-52000000.0,4000000.0,12000000.0,86000000.0,234000000.0,-52000000.0,0.0,-26000000.0,-78000000.0,-38000000.0,27000000.0,-43000000.0,-56000000.0,90000000
1,A,2017-01-31,45846,USD,2017,Q1,2017-03-08,2018-03-06,322000000.0,326000000.0,1.067000e+09,-493000000.0,574000000.0,-368000000.0,-289000000.0,-79000000.0,NaN,206000000.0,-13000000.0,-16000000.0,193000000,NaN,193000000,-25000000.0,168000000,NaN,168000000,168000000,Q1,2017-03-08,322000000.0,326000000.0,2.241000e+09,653000000.0,551000000.0,3.635000e+09,653000000.0,133000000.0,3.451000e+09,4.237000e+09,7872000000,268000000.0,190000000.0,1089000000,1.802000e+09,2.483000e+09,3572000000,5.239000e+09,0.000000e+00,-4.530000e+08,4.300000e+09,7872000000,Q1,2018-03-06,322000000.0,326000000.0,168000000.0,55000000.0,45000000.0,-152000000.0,-31000000.0,-26000000.0,9000000.0,-104000000.0,116000000.0,-32000000.0,NaN,-69000000.0,-101000000.0,-42000000.0,89000000.0,-93000000.0,-58000000.0,-48000000
2,A,2017-04-30,45846,USD,2017,Q2,2017-06-06,2018-05-31,321000000.0,325000000.0,1.102000e+09,-510000000.0,592000000.0,-391000000.0,-307000000.0,-84000000.0,NaN,201000000.0,-10000000.0,-15000000.0,191000000,NaN,191000000,-27000000.0,164000000,NaN,164000000,164000000,Q2,2017-06-06,321000000.0,325000000.0,2.389000e+09,677000000.0,548000000.0,3.800000e+09,675000000.0,134000000.0,3.407000e+09,4.216000e+09,8016000000,265000000.0,241000000.0,1187000000,1.802000e+09,2.454000e+09,3641000000,5.242000e+09,0.000000e+00,-3.930000e+08,4.375000e+09,8016000000,Q2,2018-05-31,321000000.0,325000000.0,164000000.0,54000000.0,27000000.0,12000000.0,-17000000.0,-3000000.0,-3000000.0,35000000.0,257000000.0,-43000000.0,NaN,0.0,-43000000.0,-43000000.0,52000000.0,-750

time: 46 ms (started: 2022-10-11 23:29:27 +08:00)


In [20]:
# all the price data that we have combined so far
df_prices.head()

,Ticker,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
0,A,2016-10-06,45846,47.01,46.88,47.43,47.30,45.09,NaN,1772801,324384755.0
1,A,2016-10-07,45846,47.33,46.99,47.49,47.38,45.16,NaN,1663127,324384755.0
2,A,2016-10-10,45846,47.58,47.55,48.15,47.69,45.46,NaN,1544958,324384755.0
3,A,2016-10-11,45846,47.19,45.42,47.19,45.69,43.55,NaN,2912537,324384755.0
4,A,2016-10-12,45846,45.78,45.42,45.81,45.47,43.34,NaN,1736076,324384755.0


time: 0 ns (started: 2022-10-11 23:29:27 +08:00)


In [21]:
# We want to find the price of each ticker on each Report Date. 
# To do this, we create a new dataframe using the above info.

df_report_date = df_merged[['Ticker', 'Report Date']]
df_price_date = df_prices[['Ticker', 'Date', 'Open', 'Volume']]

# In addition, we add five more columns for 5 days after reporting date
df_report_date['ReportDate+1'] = df_report_date['Report Date'] + pd.Timedelta(days=1)
df_report_date['ReportDate+2'] = df_report_date['Report Date'] + pd.Timedelta(days=2)
df_report_date['ReportDate+3'] = df_report_date['Report Date'] + pd.Timedelta(days=3)
df_report_date['ReportDate+4'] = df_report_date['Report Date'] + pd.Timedelta(days=4)
df_report_date['ReportDate+5'] = df_report_date['Report Date'] + pd.Timedelta(days=5)

time: 78 ms (started: 2022-10-11 23:29:27 +08:00)


In [22]:
# Merge the two subsets above using outer join only on the REPORT DATE
price_on_report_date = pd.merge(df_report_date, df_price_date, how='outer',
                                left_on=['Ticker','Report Date'], right_on=['Ticker','Date'])

time: 907 ms (started: 2022-10-11 23:29:28 +08:00)


In [23]:
price_on_report_date.notnull().sum()

Ticker          3188969
Report Date       41032
ReportDate+1      41032
ReportDate+2      41032
ReportDate+3      41032
ReportDate+4      41032
ReportDate+5      41032
Date            3172869
Open            3150979
Volume          3172869
dtype: int64

time: 125 ms (started: 2022-10-11 23:29:28 +08:00)


In [24]:
price_on_report_date.head(20)

,Ticker,Report Date,ReportDate+1,ReportDate+2,ReportDate+3,ReportDate+4,ReportDate+5,Date,Open,Volume
0,A,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-10-31,43.33,1556762.0
1,A,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-01-31,48.04,2552612.0
2,A,2017-04-30,2017-05-01,2017-05-02,2017-05-03,2017-05-04,2017-05-05,NaT,NaN,NaN
3,A,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-07-31,60.32,1038744.0
4,A,2017-10-31,2017-11-01,2017-11-02,2017-11-03,2017-11-04,2017-11-05,2017-10-31,67.74,1609938.0
5,A,2018-01-31,2018-02-01,2018-02-02,2018-02-03,2018-02-04,2018-02-05,2018-01-31,73.77,1949108.0
6,A,2018-04-30,2018-05-01,2018-05-02,2018-05-03,2018-05-04,2018-05-05,2018-04-30,66.49,2277705.0
7,A,2018-07-31,2018-08-01,2018-08-02,2018-08-03,2018-08-04,2018-08-05,2018-07-31,64.93,2539265.0
8,A,2018-10-31,2018-11-01,2018-11-02,2018-11-03,2018-11-04,2018-11-05,2018-10-31,64.35,3820987.0
9,A,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-01-31,75.21,2951321.0


time: 16 ms (started: 2022-10-11 23:29:29 +08:00)


As we can see from above, some reporting dates do not have price data, so we need to fill it with price data from the closest date available (either ReportDate+1 or ReportDate+2)

In [25]:
# merge the two subsets above using outer join on REPORT DATE + 1
price_on_report_date1 = pd.merge(price_on_report_date, df_price_date, how='outer',
                                left_on=['Ticker','ReportDate+1'], right_on=['Ticker','Date'])

time: 1.66 s (started: 2022-10-11 23:29:29 +08:00)


In [26]:
# merge the two subsets above using outer join on REPORT DATE + 2
price_on_report_date2 = pd.merge(price_on_report_date1, df_price_date, how='outer',
                                left_on=['Ticker','ReportDate+2'], right_on=['Ticker','Date'])

time: 2.62 s (started: 2022-10-11 23:29:30 +08:00)


In [27]:
# merge the two subsets above using outer join on REPORT DATE + 3
price_on_report_date3 = pd.merge(price_on_report_date2, df_price_date, how='outer',
                                left_on=['Ticker','ReportDate+3'], right_on=['Ticker','Date'])

time: 6.06 s (started: 2022-10-11 23:29:33 +08:00)


In [28]:
# merge the two subsets above using outer join on REPORT DATE + 4
price_on_report_date4 = pd.merge(price_on_report_date3, df_price_date, how='outer',
                                left_on=['Ticker','ReportDate+4'], right_on=['Ticker','Date'])

time: 16.5 s (started: 2022-10-11 23:29:39 +08:00)


In [29]:
# merge the two subsets above using outer join on REPORT DATE + 5
price_on_report_date5 = pd.merge(price_on_report_date4, df_price_date, how='outer',
                                left_on=['Ticker','ReportDate+5'], right_on=['Ticker','Date'])

time: 23.5 s (started: 2022-10-11 23:29:56 +08:00)


**In the final "price_on_report_date5" merged dataframe, we have managed to get the price and volume information from the closest available trading date. We will do further cleaning to simplify this.**

In [30]:
# There are some null values in the first "Open" and "Volume" columns
price_on_report_date5.head(20)

,Ticker,Report Date,ReportDate+1,ReportDate+2,ReportDate+3,ReportDate+4,ReportDate+5,Date_x,Open_x,Volume_x,Date_y,Open_y,Volume_y,Date_x,Open_x,Volume_x,Date_y,Open_y,Volume_y,Date_x,Open_x,Volume_x,Date_y,Open_y,Volume_y
0,A,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-10-31,43.33,1556762.0,2016-11-01,43.73,2133116.0,2016-11-02,43.47,2240428.0,2016-11-03,43.33,2510054.0,2016-11-04,43.69,2220908.0,NaT,NaN,NaN
1,A,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-01-31,48.04,2552612.0,2017-02-01,48.93,2244604.0,2017-02-02,48.88,1352525.0,2017-02-03,49.09,1191855.0,NaT,NaN,NaN,NaT,NaN,NaN
2,A,2017-04-30,2017-05-01,2017-05-02,2017-05-03,2017-05-04,2017-05-05,NaT,NaN,NaN,2017-05-01,55.50,1897018.0,2017-05-02,55.85,1673735.0,2017-05-03,55.68,1702565.0,2017-05-04,56.20,2609703.0,2017-05-05,56.85,1567046.0
3,A,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-07-31,60.32,1038744.0,2017-08-01,60.04,1323605.0,2017-08-02,60.00,1121493.0,2017-08-03,60.15,982876.0,2017-08-04,60.52,778762.0,NaT,NaN,NaN
4,A,2017-10-31,2017-11-01,2017-11-02,2017-11-03,2017-11-04,2017-11-05,2017-10-31,67.74,1609938.0,2017-11-01,68.20,1357307.0,2017-11-02,68.08,1854678.0,2017-11-03,68.09,1070662.0,NaT,NaN,NaN,NaT,NaN,NaN
5,A,2018-01-31,2018-02-01,2018-02-02,2018-02-03,2018-02-04,2018-02-05,2018-01-31,73.77,1949108.0,2018-02-01,73.18,1978455.0,2018-02-02,72.32,1947113.0,NaT,NaN,NaN,NaT,NaN,NaN,2018-02-05,70.86,2797258.0
6,A,2018-04-30,2018-05-01,2018-05-02,2018-05-03,2018-05-04,2018-05-05,2018-04-30,66.49,2277705.0,2018-05-01,65.63,3135146.0,2018-05-02,66.00,2240482.0,2018-05-03,65.77,2365851.0,2018-05-04,66.03,1331017.0,NaT,NaN,NaN
7,A,2018-07-31,2018-08-01,2018-08-02,2018-08-03,2018-08-04,2018-08-05,2018-07-31,64.93,2539265.0,2018-08-01,65.84,2679180.0,2018-08-02,64.75,2263190.0,2018-08-03,65.50,1857308.0,NaT,NaN,NaN,NaT,NaN,NaN
8,A,2018-10-31,2018-11-01,2018-11-02,2018-11-03,2018-11-04,2018-11-05,2018-10-31,64.35,3820987.0,2018-11-01,65.07,1882438.0,2018-11-02,65.80,2272114.0,NaT,NaN,NaN,NaT,NaN,NaN,2018-11-05,66.03,2348704.0
9,A,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-01-31,75.21,2951321.0,2019-02-01,76.16,1343915.0,NaT,NaN,NaN,NaT,NaN,NaN,2019-02-04,76.18,2245893.0,2019-02-05,75.80,1593537.0


time: 125 ms (started: 2022-10-11 23:30:19 +08:00)


In [31]:
# If there is no price on Report date, then take the price on Report date + 1.
# If there is no price on Report date + 1, then take the price on Report date + 2 and so on.
# Similarly, if there is no volume on Report date, then take the volume on Report date + 1.
# If there is no volume on Report date + 1, then take the volume on Report date + 2 and so on.

for i in range(1,6):
    price_on_report_date5.iloc[:,8] = np.where(price_on_report_date5.iloc[:,8].isnull(),
                                               price_on_report_date5.iloc[:,8+i*3],
                                               price_on_report_date5.iloc[:,8])
    
    price_on_report_date5.iloc[:,9] = np.where(price_on_report_date5.iloc[:,9].isnull(),
                                               price_on_report_date5.iloc[:,9+i*3],
                                               price_on_report_date5.iloc[:,9])
    i+=1

time: 1.11 s (started: 2022-10-11 23:30:19 +08:00)


In [32]:
# Basically, the first Open and Volume are the columns we want to keep.
# These columns should have Price and Volume data that is close to the Report Date.
price_on_report_date5.head(20)

,Ticker,Report Date,ReportDate+1,ReportDate+2,ReportDate+3,ReportDate+4,ReportDate+5,Date_x,Open_x,Volume_x,Date_y,Open_y,Volume_y,Date_x,Open_x,Volume_x,Date_y,Open_y,Volume_y,Date_x,Open_x,Volume_x,Date_y,Open_y,Volume_y
0,A,2016-10-31,2016-11-01,2016-11-02,2016-11-03,2016-11-04,2016-11-05,2016-10-31,43.33,1556762.0,2016-11-01,43.73,2133116.0,2016-11-02,43.47,2240428.0,2016-11-03,43.33,2510054.0,2016-11-04,43.69,2220908.0,NaT,NaN,NaN
1,A,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-01-31,48.04,2552612.0,2017-02-01,48.93,2244604.0,2017-02-02,48.88,1352525.0,2017-02-03,49.09,1191855.0,NaT,NaN,NaN,NaT,NaN,NaN
2,A,2017-04-30,2017-05-01,2017-05-02,2017-05-03,2017-05-04,2017-05-05,NaT,55.50,1897018.0,2017-05-01,55.50,1897018.0,2017-05-02,55.85,1673735.0,2017-05-03,55.68,1702565.0,2017-05-04,56.20,2609703.0,2017-05-05,56.85,1567046.0
3,A,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-07-31,60.32,1038744.0,2017-08-01,60.04,1323605.0,2017-08-02,60.00,1121493.0,2017-08-03,60.15,982876.0,2017-08-04,60.52,778762.0,NaT,NaN,NaN
4,A,2017-10-31,2017-11-01,2017-11-02,2017-11-03,2017-11-04,2017-11-05,2017-10-31,67.74,1609938.0,2017-11-01,68.20,1357307.0,2017-11-02,68.08,1854678.0,2017-11-03,68.09,1070662.0,NaT,NaN,NaN,NaT,NaN,NaN
5,A,2018-01-31,2018-02-01,2018-02-02,2018-02-03,2018-02-04,2018-02-05,2018-01-31,73.77,1949108.0,2018-02-01,73.18,1978455.0,2018-02-02,72.32,1947113.0,NaT,NaN,NaN,NaT,NaN,NaN,2018-02-05,70.86,2797258.0
6,A,2018-04-30,2018-05-01,2018-05-02,2018-05-03,2018-05-04,2018-05-05,2018-04-30,66.49,2277705.0,2018-05-01,65.63,3135146.0,2018-05-02,66.00,2240482.0,2018-05-03,65.77,2365851.0,2018-05-04,66.03,1331017.0,NaT,NaN,NaN
7,A,2018-07-31,2018-08-01,2018-08-02,2018-08-03,2018-08-04,2018-08-05,2018-07-31,64.93,2539265.0,2018-08-01,65.84,2679180.0,2018-08-02,64.75,2263190.0,2018-08-03,65.50,1857308.0,NaT,NaN,NaN,NaT,NaN,NaN
8,A,2018-10-31,2018-11-01,2018-11-02,2018-11-03,2018-11-04,2018-11-05,2018-10-31,64.35,3820987.0,2018-11-01,65.07,1882438.0,2018-11-02,65.80,2272114.0,NaT,NaN,NaN,NaT,NaN,NaN,2018-11-05,66.03,2348704.0
9,A,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-01-31,75.21,2951321.0,2019-02-01,76.16,1343915.0,NaT,NaN,NaN,NaT,NaN,NaN,2019-02-04,76.18,2245893.0,2019-02-05,75.80,1593537.0


time: 31 ms (started: 2022-10-11 23:30:20 +08:00)


In [33]:
# Drop off all unnecessary columns first
# Or rather keep all the necessary columns
df_reportdate = price_on_report_date5.iloc[:,[0,1,8,9]]

time: 468 ms (started: 2022-10-11 23:30:20 +08:00)


In [34]:
df_reportdate

,Ticker,Report Date,Open_x,Volume_x
0,A,2016-10-31,43.33,1556762.0
1,A,2017-01-31,48.04,2552612.0
2,A,2017-04-30,55.50,1897018.0
3,A,2017-07-31,60.32,1038744.0
4,A,2017-10-31,67.74,1609938.0
...,...,...,...,...
18926952,ZYXI,NaT,10.76,195093.0
18926953,ZYXI,NaT,10.58,252514.0
18926954,ZYXI,NaT,10.28,518118.0
18926955,ZYXI,NaT,10.01,539125.0


time: 31 ms (started: 2022-10-11 23:30:21 +08:00)


In [35]:
# Rename the columns
df_reportdate = df_reportdate.rename(columns={'Open_x':'Open',
                                            'Volume_x':'Volume'})

time: 156 ms (started: 2022-10-11 23:30:21 +08:00)


In [36]:
# Drop off all null rows without a Report Date
df_reportdate = df_reportdate[df_reportdate['Report Date'].notnull()]

time: 110 ms (started: 2022-10-11 23:30:21 +08:00)


In [37]:
# check shape
df_reportdate.shape

(41032, 4)

time: 0 ns (started: 2022-10-11 23:30:21 +08:00)


In [38]:
df_reportdate.notnull().sum()

Ticker         41032
Report Date    41032
Open           39777
Volume         39791
dtype: int64

time: 15 ms (started: 2022-10-11 23:30:21 +08:00)


In [39]:
df_merged.shape

(41032, 74)

time: 0 ns (started: 2022-10-11 23:30:21 +08:00)


## We also want to find out the stock price one year later to be able to measure performance.

In [40]:
df_reportdate['DaysLater365'] = df_reportdate['Report Date'] + pd.Timedelta(days=365)

# Adding two more columns just in case 365 days later, it is not a trading day
df_reportdate['DaysLater366'] = df_reportdate['Report Date'] + pd.Timedelta(days=366)
df_reportdate['DaysLater367'] = df_reportdate['Report Date'] + pd.Timedelta(days=367)
df_reportdate['DaysLater368'] = df_reportdate['Report Date'] + pd.Timedelta(days=368)
df_reportdate['DaysLater369'] = df_reportdate['Report Date'] + pd.Timedelta(days=369)

time: 15 ms (started: 2022-10-11 23:30:21 +08:00)


In [41]:
df_reportdate

,Ticker,Report Date,Open,Volume,DaysLater365,DaysLater366,DaysLater367,DaysLater368,DaysLater369
0,A,2016-10-31,43.33,1556762.0,2017-10-31,2017-11-01,2017-11-02,2017-11-03,2017-11-04
1,A,2017-01-31,48.04,2552612.0,2018-01-31,2018-02-01,2018-02-02,2018-02-03,2018-02-04
2,A,2017-04-30,55.50,1897018.0,2018-04-30,2018-05-01,2018-05-02,2018-05-03,2018-05-04
3,A,2017-07-31,60.32,1038744.0,2018-07-31,2018-08-01,2018-08-02,2018-08-03,2018-08-04
4,A,2017-10-31,67.74,1609938.0,2018-10-31,2018-11-01,2018-11-02,2018-11-03,2018-11-04
...,...,...,...,...,...,...,...,...,...
41027,ZYXI,2020-09-30,15.55,1115434.0,2021-09-30,2021-10-01,2021-10-02,2021-10-03,2021-10-04
41028,ZYXI,2020-12-31,12.71,318435.0,2021-12-31,2022-01-01,2022-01-02,2022-01-03,2022-01-04
41029,ZYXI,2021-03-31,13.60,423240.0,2022-03-31,2022-04-01,2022-04-02,2022-04-03,2022-04-04
41030,ZYXI,2021-06-30,14.13,144577.0,2022-06-30,2022-07-01,2022-07-02,2022-07-03,2022-07-04


time: 15 ms (started: 2022-10-11 23:30:21 +08:00)


In [42]:
# Do something similar to get the price data one year later through merging of the datasets
df_datatwodates365 = pd.merge(df_reportdate, df_price_date, how='left',
                                left_on=['Ticker','DaysLater365'], right_on=['Ticker','Date'])

# get price and volume info from 366, 367, 368 and 369 days later
df_datatwodates366 = pd.merge(df_datatwodates365, df_price_date, how='left',
                                left_on=['Ticker','DaysLater366'], right_on=['Ticker','Date'])

df_datatwodates367 = pd.merge(df_datatwodates366, df_price_date, how='left',
                                left_on=['Ticker','DaysLater367'], right_on=['Ticker','Date'])

df_datatwodates368 = pd.merge(df_datatwodates367, df_price_date, how='left',
                                left_on=['Ticker','DaysLater368'], right_on=['Ticker','Date'])

df_datatwodates369 = pd.merge(df_datatwodates368, df_price_date, how='left',
                                left_on=['Ticker','DaysLater369'], right_on=['Ticker','Date'])

time: 2.86 s (started: 2022-10-11 23:30:21 +08:00)


In [43]:
# Again, there are some missing price and volume info in the first "Open" and "Volume" column
df_datatwodates369.head(20)

,Ticker,Report Date,Open_x,Volume_x,DaysLater365,DaysLater366,DaysLater367,DaysLater368,DaysLater369,Date_x,Open_y,Volume_y,Date_y,Open_x,Volume_x,Date_x,Open_y,Volume_y,Date_y,Open_x,Volume_x,Date,Open_y,Volume_y
0,A,2016-10-31,43.33,1556762.0,2017-10-31,2017-11-01,2017-11-02,2017-11-03,2017-11-04,2017-10-31,67.74,1609938.0,2017-11-01,68.20,1357307.0,2017-11-02,68.08,1854678.0,2017-11-03,68.09,1070662.0,NaT,NaN,NaN
1,A,2017-01-31,48.04,2552612.0,2018-01-31,2018-02-01,2018-02-02,2018-02-03,2018-02-04,2018-01-31,73.77,1949108.0,2018-02-01,73.18,1978455.0,2018-02-02,72.32,1947113.0,NaT,NaN,NaN,NaT,NaN,NaN
2,A,2017-04-30,55.50,1897018.0,2018-04-30,2018-05-01,2018-05-02,2018-05-03,2018-05-04,2018-04-30,66.49,2277705.0,2018-05-01,65.63,3135146.0,2018-05-02,66.00,2240482.0,2018-05-03,65.77,2365851.0,2018-05-04,66.03,1331017.0
3,A,2017-07-31,60.32,1038744.0,2018-07-31,2018-08-01,2018-08-02,2018-08-03,2018-08-04,2018-07-31,64.93,2539265.0,2018-08-01,65.84,2679180.0,2018-08-02,64.75,2263190.0,2018-08-03,65.50,1857308.0,NaT,NaN,NaN
4,A,2017-10-31,67.74,1609938.0,2018-10-31,2018-11-01,2018-11-02,2018-11-03,2018-11-04,2018-10-31,64.35,3820987.0,2018-11-01,65.07,1882438.0,2018-11-02,65.80,2272114.0,NaT,NaN,NaN,NaT,NaN,NaN
5,A,2018-01-31,73.77,1949108.0,2019-01-31,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-01-31,75.21,2951321.0,2019-02-01,76.16,1343915.0,NaT,NaN,NaN,NaT,NaN,NaN,2019-02-04,76.18,2245893.0
6,A,2018-04-30,66.49,2277705.0,2019-04-30,2019-05-01,2019-05-02,2019-05-03,2019-05-04,2019-04-30,77.44,1726239.0,2019-05-01,78.49,1078572.0,2019-05-02,77.41,946764.0,2019-05-03,78.58,1303198.0,NaT,NaN,NaN
7,A,2018-07-31,64.93,2539265.0,2019-07-31,2019-08-01,2019-08-02,2019-08-03,2019-08-04,2019-07-31,70.19,3846063.0,2019-08-01,69.63,2494123.0,2019-08-02,69.12,2803508.0,NaT,NaN,NaN,NaT,NaN,NaN
8,A,2018-10-31,64.35,3820987.0,2019-10-31,2019-11-01,2019-11-02,2019-11-03,2019-11-04,2019-10-31,76.00,970472.0,2019-11-01,76.51,1106264.0,NaT,NaN,NaN,NaT,NaN,NaN,2019-11-04,77.68,1010174.0
9,A,2019-01-31,75.21,2951321.0,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-01-31,84.03,2850446.0,NaT,NaN,NaN,NaT,NaN,NaN,2020-02-03,83.29,1919771.0,2020-02-04,82.85,1676020.0


time: 15 ms (started: 2022-10-11 23:30:24 +08:00)


In [44]:
for i in range(1,5):
    df_datatwodates369.iloc[:,10] = np.where(df_datatwodates369.iloc[:,10].isnull(),
                                               df_datatwodates369.iloc[:,10+i*3],
                                               df_datatwodates369.iloc[:,10])
    
    df_datatwodates369.iloc[:,11] = np.where(df_datatwodates369.iloc[:,11].isnull(),
                                               df_datatwodates369.iloc[:,11+i*3],
                                               df_datatwodates369.iloc[:,11])
    i+=1

time: 16 ms (started: 2022-10-11 23:30:24 +08:00)


In [45]:
df_datatwodates369.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41032 entries, 0 to 41031
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Ticker        41032 non-null  object        
 1   Report Date   41032 non-null  datetime64[ns]
 2   Open_x        39777 non-null  float64       
 3   Volume_x      39791 non-null  float64       
 4   DaysLater365  41032 non-null  datetime64[ns]
 5   DaysLater366  41032 non-null  datetime64[ns]
 6   DaysLater367  41032 non-null  datetime64[ns]
 7   DaysLater368  41032 non-null  datetime64[ns]
 8   DaysLater369  41032 non-null  datetime64[ns]
 9   Date_x        20252 non-null  datetime64[ns]
 10  Open_y        31066 non-null  float64       
 11  Volume_y      31080 non-null  float64       
 12  Date_y        21344 non-null  datetime64[ns]
 13  Open_x        21338 non-null  float64       
 14  Volume_x      21344 non-null  float64       
 15  Date_x        24184 non-null  dateti

In [46]:
# Drop off all unnecessary columns first
# Or rather keep all the necessary columns
df_datatwodates = df_datatwodates369.iloc[:,[0,1,2,3,4,10,11]]
df_datatwodates

,Ticker,Report Date,Open_x,Volume_x,DaysLater365,Open_y,Volume_y
0,A,2016-10-31,43.33,1556762.0,2017-10-31,67.74,1609938.0
1,A,2017-01-31,48.04,2552612.0,2018-01-31,73.77,1949108.0
2,A,2017-04-30,55.50,1897018.0,2018-04-30,66.49,2277705.0
3,A,2017-07-31,60.32,1038744.0,2018-07-31,64.93,2539265.0
4,A,2017-10-31,67.74,1609938.0,2018-10-31,64.35,3820987.0
...,...,...,...,...,...,...,...
41027,ZYXI,2020-09-30,15.55,1115434.0,2021-09-30,10.58,252514.0
41028,ZYXI,2020-12-31,12.71,318435.0,2021-12-31,NaN,NaN
41029,ZYXI,2021-03-31,13.60,423240.0,2022-03-31,NaN,NaN
41030,ZYXI,2021-06-30,14.13,144577.0,2022-06-30,NaN,NaN


time: 16 ms (started: 2022-10-11 23:30:24 +08:00)


We only draw data from a 5-year range between Oct 2016 to Oct 2021. So we can see that there are some data misssing for price data one year later in the Open_y and Volume_y columns.

In [47]:
df_datatwodates = df_datatwodates.rename(columns={'Open_x':'Open1',
                                                  'Volume_x':'Volume1',
                                                  'Open_y':'Open2',
                                                  'Volume_y':'Volume2',
                                                  'Report Date':'Date1',
                                                  'DaysLater365':'Date2'})

time: 0 ns (started: 2022-10-11 23:30:24 +08:00)


In [48]:
df_datatwodates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41032 entries, 0 to 41031
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Ticker   41032 non-null  object        
 1   Date1    41032 non-null  datetime64[ns]
 2   Open1    39777 non-null  float64       
 3   Volume1  39791 non-null  float64       
 4   Date2    41032 non-null  datetime64[ns]
 5   Open2    31066 non-null  float64       
 6   Volume2  31080 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(1)
memory usage: 2.5+ MB
time: 0 ns (started: 2022-10-11 23:30:25 +08:00)


In [49]:
df_datatwodates[df_datatwodates.isnull().any(axis=1)]

,Ticker,Date1,Open1,Volume1,Date2,Open2,Volume2
16,A,2020-10-31,103.52,1572506.0,2021-10-31,NaN,NaN
17,A,2021-01-31,120.69,1292732.0,2022-01-31,NaN,NaN
18,A,2021-04-30,133.90,1514162.0,2022-04-30,NaN,NaN
19,A,2021-07-31,154.05,1438321.0,2022-07-31,NaN,NaN
36,AA,2020-12-31,23.00,3443724.0,2021-12-31,NaN,NaN
...,...,...,...,...,...,...,...
41012,ZYNE,2021-09-30,4.25,761190.0,2022-09-30,NaN,NaN
41028,ZYXI,2020-12-31,12.71,318435.0,2021-12-31,NaN,NaN
41029,ZYXI,2021-03-31,13.60,423240.0,2022-03-31,NaN,NaN
41030,ZYXI,2021-06-30,14.13,144577.0,2022-06-30,NaN,NaN


time: 15 ms (started: 2022-10-11 23:30:25 +08:00)


# Saving our files to csv

In [50]:
# We save df_datatwodates as y first (total 41,032 rows), even though it has some null values
y = df_datatwodates
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41032 entries, 0 to 41031
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Ticker   41032 non-null  object        
 1   Date1    41032 non-null  datetime64[ns]
 2   Open1    39777 non-null  float64       
 3   Volume1  39791 non-null  float64       
 4   Date2    41032 non-null  datetime64[ns]
 5   Open2    31066 non-null  float64       
 6   Volume2  31080 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(1)
memory usage: 2.5+ MB
time: 15 ms (started: 2022-10-11 23:30:25 +08:00)


In [51]:
# We also save df_merged as our x dataframe (total 41,032 rows), so it contains all our fundamental data.
x = df_merged
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41032 entries, 0 to 41031
Data columns (total 74 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   Ticker                                           41032 non-null  object        
 1   Report Date                                      41032 non-null  datetime64[ns]
 2   SimFinId                                         41032 non-null  int64         
 3   Currency                                         41032 non-null  object        
 4   Fiscal Year                                      41032 non-null  int64         
 5   Fiscal Period_x                                  41032 non-null  object        
 6   Publish Date                                     41032 non-null  datetime64[ns]
 7   Restated Date_x                                  41032 non-null  datetime64[ns]
 8   Shares (Basic)_x                    

In [52]:
# save the final file to csv
x.to_csv("./data/01_fundamentals.csv")
y.to_csv("./data/01_pricedata.csv")

time: 3.25 s (started: 2022-10-11 23:30:25 +08:00)


In [53]:
pickle.dump(x, open('./data/01_fundamentals.pkl', 'wb'))
pickle.dump(y, open('./data/01_pricedata.pkl', 'wb'))

time: 31 ms (started: 2022-10-11 23:30:28 +08:00)
